# Data Extraction

#### Extracting datafile from Kaggle and storing it in stagging dataframe


In [36]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi

In [7]:
api=KaggleApi()
api.authenticate()
dataset="aungpyaeap/supermarket-sales"
api.dataset_download_files(dataset,path='data',unzip=True)

Dataset URL: https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales


In [96]:
stage_df=pd.read_csv("data/supermarket_sales.csv")

In [98]:
stage_df.head(5)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


# Data Transformation and Loading

### Creating dimension dataframes


##### Customer Type Dimension - 

In [118]:
customer_dim=stage_df[["Customer type"]].drop_duplicates()

In [120]:
customer_dim.head()

,Customer type
0,Member
1,Normal


In [224]:
## Inserting unique values to customer_type_id column 

customer_dim["customer_type_id"]=customer_dim.index+101

In [226]:
customer_dim.head()

,Customer type,customer_type_id
0,Member,101
1,Normal,102


##### Product line Dimension -

In [192]:
product_line_dim=stage_df[["Product line"]].drop_duplicates()
product_line_dim.reset_index(drop=True,inplace=True)

In [147]:
product_line_dim.head()

,Product line
0,Health and beauty
1,Electronic accessories
2,Home and lifestyle
3,Sports and travel
4,Food and beverages


In [268]:
## Inserting unique values to product_line_id column
product_line_dim["product_line_id"]=product_line_dim.index+1

In [270]:
product_line_dim.head()

,Product line,product_line_id
0,Health and beauty,1
1,Electronic accessories,2
2,Home and lifestyle,3
3,Sports and travel,4
4,Food and beverages,5


### Creating Fact dataframe



In [272]:
stage_df2=stage_df.merge(product_line_dim,how='left',on='Product line')

In [274]:
stage_df2=stage_df2.merge(customer_dim,how='left',on='Customer type')

In [280]:
sales_fact=stage_df2[['customer_type_id','product_line_id','Branch','Gender','Invoice ID','Total','Date','Payment']]

In [282]:
sales_fact.head()

,customer_type_id,product_line_id,Branch,Gender,Invoice ID,Total,Date,Payment
0,101,1,A,Female,750-67-8428,548.9715,1/5/2019,Ewallet
1,102,2,C,Female,226-31-3081,80.2200,3/8/2019,Cash
2,102,3,A,Male,631-41-3108,340.5255,3/3/2019,Credit card
3,101,1,A,Male,123-19-1176,489.0480,1/27/2019,Ewallet
4,102,4,A,Male,373-73-7910,634.3785,2/8/2019,Ewallet
